# USD Correlation Analysis

In [1]:
# Import libraries and dependencies
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import datetime


%matplotlib inline

In [2]:
#Multiple Ticker Analysis

tickers = ['UUP', 'DBO', 'DBC', 'SPY', 'GLD']



In [3]:
#Download stock data
stock_data = yf.download(tickers,start="2018-01-01")
# Use the `drop` function with the `level` parameter to drop extra columns in the multi-index DataFrame

stock_data = stock_data.drop(columns=["Close", "High", "Low", "Open", "Volume"])
stock_data.head()

[*********************100%***********************]  5 of 5 completed


Adj Close                                              
                  DBC        DBO         GLD         SPY        UUP
Date                                                               
2018-01-02  16.216225   9.866567  125.150002  256.217468  23.210712
2018-01-03  16.313387  10.079584  124.820000  257.838104  23.298008
2018-01-04  16.293955  10.098950  125.459999  258.924835  23.220413
2018-01-05  16.235657  10.050536  125.330002  260.650299  23.239809
2018-01-08  16.225943  10.108632  125.309998  261.126984  23.346504

In [4]:
# calculate daily returns
stock_returns = stock_data.pct_change()
stock_returns = stock_returns.dropna()
stock_returns.columns = stock_returns.columns.droplevel()
stock_returns

,DBC,DBO,GLD,SPY,UUP
Date,,,,,
2018-01-03,0.005992,0.021590,-0.002637,0.006325,0.003761
2018-01-04,-0.001191,0.001921,0.005127,0.004215,-0.003331
2018-01-05,-0.003578,-0.004794,-0.001036,0.006664,0.000835
2018-01-08,-0.000598,0.005780,-0.000160,0.001829,0.004591
2018-01-09,0.006587,0.017241,-0.004628,0.002263,0.002077
...,...,...,...,...,...
2020-09-03,-0.008969,-0.007926,-0.008104,-0.034414,0.000399
2020-09-04,-0.009050,-0.034621,0.002760,-0.008165,0.001597
2020-09-08,-0.024353,-0.057931,-0.001927,-0.027323,0.005181


In [5]:
# Initial correlation table
correlation_table = stock_returns.corr()
correlation_table

,DBC,DBO,GLD,SPY,UUP
DBC,1.000000,0.921314,0.171283,0.532639,-0.062902
DBO,0.921314,1.000000,0.011235,0.463710,0.006978
GLD,0.171283,0.011235,1.000000,0.044326,-0.421831
SPY,0.532639,0.463710,0.044326,1.000000,0.080181
UUP,-0.062902,0.006978,-0.421831,0.080181,1.000000


In [6]:
#calculating a rolling 30-day return
cumulative_returns_df = (stock_returns+1).rolling(30).apply(np.prod)-1
cumulative_returns_df

,DBC,DBO,GLD,SPY,UUP
Date,,,,,
2018-01-03,NaN,NaN,NaN,NaN,NaN
2018-01-04,NaN,NaN,NaN,NaN,NaN
2018-01-05,NaN,NaN,NaN,NaN,NaN
2018-01-08,NaN,NaN,NaN,NaN,NaN
2018-01-09,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...
2020-09-03,0.029503,0.021769,0.022350,0.069451,-0.021867
2020-09-04,0.018605,-0.016282,0.016452,0.067595,-0.016078
2020-09-08,-0.015361,-0.081989,-0.005158,0.030908,-0.003556


In [7]:
# Historical Cross Asset Correlation on rolling 30 day returns
correlation_table = cumulative_returns_df.corr()
correlation_table.columns = ['Invesco DB Commodity Index Tracking Fund (DBC)', 'Invesco DB Oil Fund (DBO)', 'GOLD', 'S&P500 (SPY)', 'USD (UUP)']
correlation_table.style.set_caption("Historical Cross Asset Correlation on rolling 30 day returns")

,Invesco DB Commodity Index Tracking Fund (DBC),Invesco DB Oil Fund (DBO),GOLD,S&P500 (SPY),USD (UUP)
DBC,1.000000,0.959947,0.039899,0.707274,-0.415845
DBO,0.959947,1.000000,-0.088712,0.657474,-0.359574
GLD,0.039899,-0.088712,1.000000,0.174704,-0.453321
SPY,0.707274,0.657474,0.174704,1.000000,-0.361686
UUP,-0.415845,-0.359574,-0.453321,-0.361686,1.000000


In [8]:
# Rolling 30 day correlations over the last 15 days
correlation_table_15 = cumulative_returns_df[-15:].corr()
correlation_table_15.columns = ['Invesco DB Commodity Index Tracking Fund (DBC)', 'Invesco DB Oil Fund (DBO)', 'GOLD', 'S&P500 (SPY)', 'USD (UUP)']
correlation_table_15.style.set_caption("Rolling 30 day correlations over the last 15 days")

,Invesco DB Commodity Index Tracking Fund (DBC),Invesco DB Oil Fund (DBO),GOLD,S&P500 (SPY),USD (UUP)
DBC,1.000000,0.986282,0.970291,0.856796,-0.979371
DBO,0.986282,1.000000,0.950649,0.853946,-0.973524
GLD,0.970291,0.950649,1.000000,0.790492,-0.981717
SPY,0.856796,0.853946,0.790492,1.000000,-0.818853
UUP,-0.979371,-0.973524,-0.981717,-0.818853,1.000000


In [9]:
# Rolling 30 day correlations over the last 30 days
correlation_table_30 = cumulative_returns_df[-30:].corr()
correlation_table_30.columns = ['Invesco DB Commodity Index Tracking Fund (DBC)', 'Invesco DB Oil Fund (DBO)', 'GOLD', 'S&P500 (SPY)', 'USD (UUP)']
correlation_table_30.style.set_caption("Rolling 30 day correlations over the last 30 days")

,Invesco DB Commodity Index Tracking Fund (DBC),Invesco DB Oil Fund (DBO),GOLD,S&P500 (SPY),USD (UUP)
DBC,1.000000,0.975660,0.824876,0.736145,-0.925389
DBO,0.975660,1.000000,0.772448,0.720081,-0.934255
GLD,0.824876,0.772448,1.000000,0.362020,-0.868017
SPY,0.736145,0.720081,0.362020,1.000000,-0.540699
UUP,-0.925389,-0.934255,-0.868017,-0.540699,1.000000


In [10]:
import xlsxwriter

In [11]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('USD_correlations_rolling_30day_returns.xlsx', engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
correlation_table.to_excel(writer, sheet_name='Historical', float_format='%.2f')
correlation_table_15.to_excel(writer, sheet_name='last 15 days', float_format='%.2f')
correlation_table_30.to_excel(writer, sheet_name='last 30 days', float_format='%.2f')


# Close the Pandas Excel writer and output the Excel file.
writer.save()